# Feature extraction in COGNITION_v2
some critical features

200 Hz

broadband [0,40]

In [1]:
#external libraries
import os
import dotenv
import pandas as pd
import numpy as np
from scipy import signal as sg
import pickle as pkl
import time
import matplotlib.pyplot as plt

#project library
from spinco import *

#environment variables
dotenv.load_dotenv('lab.env')

#project variables
datapath=os.environ['DATAPATH']

In [2]:
#define features path
cognipath=datapath+"/COGNITION_v2"
featurespath=cognipath+"/features"
        
windowDurations=[0.5,1,1.5,2]

## Load data

In [3]:
signals, annotations, signalsMetadata = loadCOGNITIONSpindles_v2(cognipath,returnSignals=True)

In [4]:
signalsMetadata

,filename,subjectId,channel,duration,samplerate,isOriginalSamplerate,database
0,COG001_F3.pkl,COG001,F3,39600.0,200,False,COGNITION_v2
1,COG001_C3.pkl,COG001,C3,39600.0,200,False,COGNITION_v2
2,COG001_F4.pkl,COG001,F4,39600.0,200,False,COGNITION_v2
3,COG001_C4.pkl,COG001,C4,39600.0,200,False,COGNITION_v2
4,COG002_F3.pkl,COG002,F3,39600.0,200,False,COGNITION_v2
...,...,...,...,...,...,...,...
71,COG018_C4.pkl,COG018,C4,39600.0,200,False,COGNITION_v2
72,COG019_F3.pkl,COG019,F3,41400.0,200,False,COGNITION_v2
73,COG019_C3.pkl,COG019,C3,41400.0,200,False,COGNITION_v2
74,COG019_F4.pkl,COG019,F4,41400.0,200,False,COGNITION_v2


## Preprocess

In [5]:
samplerate=200

broadband in 0-40Hz, previous experiments (up tp 50Hz, are now in folders as .../features_old/...)

In [6]:
def preprocessVector(vector,samplerate):
    #1. Lowpass 40Hz
    vector=filterBand(vector,[0,40],samplerate,filterOrder=4)
    #2. Z-score
    m=np.mean(vector)
    s=np.std(vector)
    vector=(vector-m)/s
    return vector

In [7]:
for subject,subject_signals in signals.items():
    for channel, signal in subject_signals.items():
        signals[subject][channel]=preprocessVector(signal,samplerate)

## Extract features

### folder estructure

In [8]:
fspath=featurespath+'/'+str(samplerate)+'fs/'
if not os.path.isdir(fspath):
    os.mkdir(fspath)
for window in windowDurations:
    windowPath=fspath+str(window)+'win'
    if not os.path.isdir(windowPath):
        os.mkdir(windowPath)
    for subject in signalsMetadata.subjectId:
        subjectPath=windowPath+'/'+subject
        if not os.path.isdir(subjectPath):
            os.mkdir(subjectPath)
            for channel in signalsMetadata.channel:
                channelPath=subjectPath+'/'+channel
                if not os.path.isdir(channelPath):
                    os.mkdir(channelPath)

### band definition

In [9]:
bands={
    'delta1':[0.1,2],
    'delta2':[2,4],
    'theta':[4,8],
    'alpha':[8,13],
    'sigma':[11,16],
    'beta1':[13,19],
    'beta2':[19,30]
    }

### computation

In [10]:
for window in windowDurations:
    windowPath=featurespath+'/'+str(samplerate)+'fs/'+str(window)
    for ind, row in signalsMetadata.iterrows():
        subject=row.subjectId
        subjectPath=windowPath+'/'+subject
        channel=row.channel
        channelPath=subjectPath+'/'+channel
        signal=signals[subject][channel]
        #need to define the time vector for each signal:
        timepoints=np.arange(len(signal))/samplerate
        
        #6. Hjort Activity
        characteristic='hjortActivity'
        bandName='broadband'
        filepath=getFilepath_v2(window,subject,channel,characteristic,bandName,samplerate,featurespath)
        if not os.path.exists(filepath):
            aux=hjortActivity(signal,window,samplerate)
            saveFeature_v2(aux,window,subject,channel,characteristic,bandName,samplerate,featurespath)
        for bandName, band in bands.items():
            filepath=getFilepath_v2(window,subject,channel,characteristic,bandName,samplerate,featurespath)
            if not os.path.exists(filepath):
                filtered=filterBand(signal,band,samplerate)
                aux=hjortActivity(filtered,window,samplerate)
                saveFeature_v2(aux,window,subject,channel,characteristic,bandName,samplerate,featurespath)
        #7. Hjort Mobility
        characteristic='hjortMobility'
        bandName='broadband'
        filepath=getFilepath_v2(window,subject,channel,characteristic,bandName,samplerate,featurespath)
        if not os.path.exists(filepath):
            aux=hjortMobility(signal,timepoints,window,samplerate)
            saveFeature_v2(aux,window,subject,channel,characteristic,bandName,samplerate,featurespath)
        for bandName, band in bands.items():
            filepath=getFilepath_v2(window,subject,channel,characteristic,bandName,samplerate,featurespath)
            if not os.path.exists(filepath):
                filtered=filterBand(signal,band,samplerate)
                aux=hjortMobility(filtered,timepoints,window,samplerate)
                saveFeature_v2(aux,window,subject,channel,characteristic,bandName,samplerate,featurespath)
        #8. Hjort Complexity
        characteristic='hjortComplexity'
        bandName='broadband'
        filepath=getFilepath_v2(window,subject,channel,characteristic,bandName,samplerate,featurespath)
        if not os.path.exists(filepath):
            aux=hjortComplexity(signal,timepoints,window,samplerate)
            saveFeature_v2(aux,window,subject,channel,characteristic,bandName,samplerate,featurespath)
        for bandName, band in bands.items():
            filepath=getFilepath_v2(window,subject,channel,characteristic,bandName,samplerate,featurespath)
            if not os.path.exists(filepath):
                filtered=filterBand(signal,band,samplerate)
                aux=hjortComplexity(filtered,timepoints,window,samplerate)
                saveFeature_v2(aux,window,subject,channel,characteristic,bandName,samplerate,featurespath)
        #9. PetrosianFractalDimension
        characteristic='petrosian'
        bandName='broadband'
        filepath=getFilepath_v2(window,subject,channel,characteristic,bandName,samplerate,featurespath)
        if not os.path.exists(filepath):
            aux=petrosianFractalDimension(signal,timepoints,window,samplerate)
            saveFeature_v2(aux,window,subject,channel,characteristic,bandName,samplerate,featurespath)
        for bandName, band in bands.items():
            filepath=getFilepath_v2(window,subject,channel,characteristic,bandName,samplerate,featurespath)
            if not os.path.exists(filepath):
                filtered=filterBand(signal,band,samplerate)
                aux=petrosianFractalDimension(filtered,timepoints,window,samplerate)
                saveFeature_v2(aux,window,subject,channel,characteristic,bandName,samplerate,featurespath)
        
        #12. sigma index
        characteristic='sigmaIndex'
        bandName='broadband'
        filepath=getFilepath_v2(window,subject,channel,characteristic,bandName,samplerate,featurespath)
        if not os.path.exists(filepath):
            aux=sigmaindex(signal,window,samplerate)
            saveFeature_v2(aux,window,subject,channel,characteristic,bandName,samplerate,featurespath)
            

saving feature: D:/SpinCo/COGNITION_v2/features/200fs/2win/COG007/C4/2_COG007_C4_petrosian_theta.fd
saving feature: D:/SpinCo/COGNITION_v2/features/200fs/2win/COG007/C4/2_COG007_C4_petrosian_alpha.fd
saving feature: D:/SpinCo/COGNITION_v2/features/200fs/2win/COG007/C4/2_COG007_C4_petrosian_sigma.fd
saving feature: D:/SpinCo/COGNITION_v2/features/200fs/2win/COG007/C4/2_COG007_C4_petrosian_beta1.fd
saving feature: D:/SpinCo/COGNITION_v2/features/200fs/2win/COG007/C4/2_COG007_C4_petrosian_beta2.fd
saving feature: D:/SpinCo/COGNITION_v2/features/200fs/2win/COG007/C4/2_COG007_C4_sigmaIndex_broadband.fd
saving feature: D:/SpinCo/COGNITION_v2/features/200fs/2win/COG008/F3/2_COG008_F3_hjortActivity_broadband.fd
saving feature: D:/SpinCo/COGNITION_v2/features/200fs/2win/COG008/F3/2_COG008_F3_hjortActivity_delta1.fd
saving feature: D:/SpinCo/COGNITION_v2/features/200fs/2win/COG008/F3/2_COG008_F3_hjortActivity_delta2.fd
saving feature: D:/SpinCo/COGNITION_v2/features/200fs/2win/COG008/F3/2_COG008

C:\Users\Gurdiel\Code\SpinCo\src\spinco.py:333: RuntimeWarning: divide by zero encountered in divide
  mobility=np.sqrt(moveVarCentered(vectorGradient,windowSampleCount)/moveVarCentered(vector,windowSampleCount))
C:\Users\Gurdiel\Code\SpinCo\src\spinco.py:333: RuntimeWarning: invalid value encountered in divide
  mobility=np.sqrt(moveVarCentered(vectorGradient,windowSampleCount)/moveVarCentered(vector,windowSampleCount))


saving feature: D:/SpinCo/COGNITION_v2/features/200fs/2win/COG010/F3/2_COG010_F3_hjortMobility_beta1.fd
saving feature: D:/SpinCo/COGNITION_v2/features/200fs/2win/COG010/F3/2_COG010_F3_hjortMobility_beta2.fd
saving feature: D:/SpinCo/COGNITION_v2/features/200fs/2win/COG010/F3/2_COG010_F3_hjortComplexity_broadband.fd
saving feature: D:/SpinCo/COGNITION_v2/features/200fs/2win/COG010/F3/2_COG010_F3_hjortComplexity_delta1.fd
saving feature: D:/SpinCo/COGNITION_v2/features/200fs/2win/COG010/F3/2_COG010_F3_hjortComplexity_delta2.fd
saving feature: D:/SpinCo/COGNITION_v2/features/200fs/2win/COG010/F3/2_COG010_F3_hjortComplexity_theta.fd
saving feature: D:/SpinCo/COGNITION_v2/features/200fs/2win/COG010/F3/2_COG010_F3_hjortComplexity_alpha.fd
saving feature: D:/SpinCo/COGNITION_v2/features/200fs/2win/COG010/F3/2_COG010_F3_hjortComplexity_sigma.fd
saving feature: D:/SpinCo/COGNITION_v2/features/200fs/2win/COG010/F3/2_COG010_F3_hjortComplexity_beta1.fd
saving feature: D:/SpinCo/COGNITION_v2/featu